In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import h5py

from tqdm import tqdm


In [2]:
df = pd.read_parquet('/Users/diego/Documents/tesis_goes16/inputs/absolute_paths_h5.parquet')
df

,absolute_paths
0,/Users/diego/Documents/tesis_goes16/inputs/tru...
1,/Users/diego/Documents/tesis_goes16/inputs/tru...
2,/Users/diego/Documents/tesis_goes16/inputs/tru...
3,/Users/diego/Documents/tesis_goes16/inputs/tru...
4,/Users/diego/Documents/tesis_goes16/inputs/tru...
...,...
104727,/Users/diego/Documents/tesis_goes16/inputs/tru...
104728,/Users/diego/Documents/tesis_goes16/inputs/tru...
104729,/Users/diego/Documents/tesis_goes16/inputs/tru...
104730,/Users/diego/Documents/tesis_goes16/inputs/tru...


In [3]:
sample_h5=df['absolute_paths'].iloc[0]

In [4]:
sample_h5

'/Users/diego/Documents/tesis_goes16/inputs/trusted2/2023_07_09_20_59_59.h5'

In [5]:
with h5py.File(sample_h5, 'r') as f:
    # Listar todos los datasets y grupos en la raíz
    for key in f.keys():
        print(key)

    # Acceder a un dataset específico
    data = f['rainfall'][:]
    print(data)

rainfall
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.01574084 ... 0.         0.         0.        ]
 [0.         0.         0.05995937 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [6]:
data.shape

(128, 128)

In [8]:
# Supongamos que 'df' es un DataFrame que contiene los paths absolutos de los archivos .h5
# Ejemplo: df = pd.DataFrame({'paths': ['/path/to/file1.h5', '/path/to/file2.h5']})
file_paths = df['absolute_paths'].tolist()

# Nombres de los archivos HDF5 finales
output_file = '/Users/diego/Documents/tesis_goes16/inputs/combined_data_final.h5'
output_file_32_32 = '/Users/diego/Documents/tesis_goes16/inputs/combined_data_32_32.h5'

# Crear los archivos HDF5 finales
with h5py.File(output_file, 'w') as h5_final, h5py.File(output_file_32_32, 'w') as h5_32_32:
    # Usar tqdm para mostrar el progreso
    for file_path in tqdm(file_paths, desc="Procesando archivos", unit="archivo"):
        # Extraer el nombre del archivo sin la extensión
        key_name = os.path.splitext(os.path.basename(file_path))[0]
        
        # Abrir el archivo HDF5 y leer el array bajo la llave 'rainfall'
        with h5py.File(file_path, 'r') as h5_file:
            if 'rainfall' in h5_file:
                data = h5_file['rainfall'][:]
                
                # Guardar el array completo en el archivo HDF5 final
                h5_final.create_dataset(key_name, data=data)
                
                # Extraer el cuadrado de 32x32 píxeles más interno
                center_32_32 = data[
                    data.shape[0] // 2 - 16 : data.shape[0] // 2 + 16,
                    data.shape[1] // 2 - 16 : data.shape[1] // 2 + 16
                ]
                
                # Guardar el cuadrado 32x32 en el archivo HDF5 correspondiente
                h5_32_32.create_dataset(key_name, data=center_32_32)
            else:
                print(f"Warning: 'rainfall' key not found in {file_path}")

print(f"Archivo combinado creado en: {output_file}")
print(f"Archivo combinado 32x32 creado en: {output_file_32_32}")

Procesando archivos: 100%|██████████| 104732/104732 [01:55<00:00, 905.48archivo/s] 


Archivo combinado creado en: /Users/diego/Documents/tesis_goes16/inputs/combined_data_final.h5
Archivo combinado 32x32 creado en: /Users/diego/Documents/tesis_goes16/inputs/combined_data_32_32.h5
